In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats
from scipy.optimize import curve_fit
import math

pd.options.plotting.backend = 'plotly'

# Ventilation and Deposition

In [ ]:
vd_filename = 'data_mesh/vd.edf'

if vd_filename.endswith('.edf'):
    df = pd.read_csv(vd_filename, sep='\t', skiprows=9)
    df.rename(columns = {'Epoch_UTC': 'time'}, inplace=True)
    # set time off of zero rather than epoch
    time_offset = df.time[0]
    df['time'] -= time_offset
    col_fit = 'MassConc_2p5_SPS3x_AE8B4EE783812236'
else:
    df = pd.read_csv(vd_filename)
    print('F°: {}'.format(df['temp_F'][10]))
    print('C°: {}'.format(df['temp_C'][10]))
    print('rh: {}%'.format(df.rh[10]))

    # column to fit on, mass PM2.5 is a good choice for NaCl aerosol CADR
    # changing this variable makes it possible to explore other options
    col_fit = 'mPM2.5'
    # set fit_under to the high value where the fit starts. for mPM2.5 use 1000, mPM1.0 use 450.
    fit_under = 1000

# get rid of all the columns not used to fit the data
for col in [x for x in df.columns.values.tolist() if x not in [col_fit, 'time']]:
    del df[col]

# especially not bothering with pm4.0 and pm10 bc they are just interpolations of 0.5, 1.0, and 2.5
# see sensirion docs for the explanation

# plot the original data
df.set_index('time').plot(title='Ventilation and Deposition', kind='line')


In [ ]:

# see equation (S1) from supplemental material here: https://www.tandfonline.com/doi/full/10.1080/02786826.2022.2054674?scroll=top&needAccess=true
# version of the functions where ACH is unknown and has to be solved for
class DecayFuncs:
    def __init__(self, C_bgd, C_pt0):
        ## logarithmic function
        def func(t, ACH):
            # divide by 3600 to convert seconds to hours
            return C_bgd + C_pt0 * np.exp(-ACH*t / 3600)

        # linear version of the same function, it's the natural log of the exponential function
        def linear_func(t, ACH):
            return np.log(C_pt0) - ACH * t / 3600

        self.func = func
        self.linear_func = linear_func

# perform the curve fitting and return the resulting parameters
def test_fit(df):
    C_bgd = 0
    C_pt0 = df[col_fit][df.index[0]]
    f = DecayFuncs(C_bgd, C_pt0)

    popt, pcov = curve_fit(f.linear_func, df.time, np.log(df[col_fit]))
    stderr = np.sqrt(np.diag(pcov)[0])
    return (C_pt0, popt, stderr)

Find ACH_vd for this run based on the natural decay period.

In [ ]:
# begin window search at pm2.5 max
pm25_max_idx = df.idxmax()[col_fit]
if pm25_max_idx != 0:
    df = df.tail(-pm25_max_idx).copy()
    df.index -= df.index[0]
    df['time'] -= df.time[0]

# analyze CADR in window of [1000, 100] because error of sps 30 is multiplicative in this range, +/- 10%
# in [100, 0] error becomes +/- 10 which means error will become exaggerated at these levels
# see: https://sensirion.com/media/documents/8600FF88/616542B5/Sensirion_PM_Sensors_Datasheet_SPS30.pdf

# cut off all records before pm2.5 dropped to 1000
drop_start_idx = df[df[col_fit] <= fit_under].index[0]
if drop_start_idx != 0:
    df = df.tail(-drop_start_idx).copy()
    df.index -= df.index[0]
    df['time'] -= df.time[0]

lt25_iloc = df[df[col_fit] < 100].index[0]
df = df.head(lt25_iloc).copy()

print('num data points to fit: {}'.format(len(df)))

C_pt0, popt, naturaldecay_stderr = test_fit(df)
ACH_vd = popt[0]
print('C_pt0: {}'.format(C_pt0))
print('ACH_vd: {}'.format(ACH_vd))
print('stderr: {}'.format(naturaldecay_stderr))

f = DecayFuncs(0, C_pt0)

plt.scatter(df.time, df[col_fit], label="Original Data")
plt.plot(df.time, f.func(df.time, ACH_vd), 'r-', label="ACH_vd {:0.2f} ±{:0.2f}".format(ACH_vd, naturaldecay_stderr))
plt.legend()


# Trial

In [ ]:
filename = 'data_mesh/trial1.edf'

if filename.endswith('.edf'):
    df = pd.read_csv(filename, sep='\t', skiprows=9)
    df.rename(columns = {'Epoch_UTC': 'time'}, inplace=True)
    # set time off of zero rather than epoch
    time_offset = df.time[0]
    df['time'] -= time_offset
else:
    df = pd.read_csv(filename)
    print('F°: {}'.format(df['temp_F'][10]))
    print('C°: {}'.format(df['temp_C'][10]))
    print('rh: {}%'.format(df.rh[10]))

# get rid of all the columns not used to fit the data
for col in [x for x in df.columns.values.tolist() if x not in [col_fit, 'time']]:
    del df[col]

# especially not bothering with pm4.0 and pm10 bc they are just interpolations of 0.5, 1.0, and 2.5
# see sensirion docs for the explanation

# plot the original data
df.set_index('time').plot(title='Trial', kind='line')


In [ ]:
# begin window search at pm2.5 max
pm25_max_idx = df.idxmax()[col_fit]
if pm25_max_idx != 0:
    df = df.tail(-pm25_max_idx).copy()
    df.index -= df.index[0]
    df['time'] -= df.time[0]

# analyze CADR in window of [1000, 100] because error of sps 30 is multiplicative in this range, +/- 10%
# in [100, 0] error becomes +/- 10 which means error will become exaggerated at these levels
# see: https://sensirion.com/media/documents/8600FF88/616542B5/Sensirion_PM_Sensors_Datasheet_SPS30.pdf

# cut off all records before pm2.5 dropped to 1000
drop_start_idx = df[df[col_fit] <= fit_under].index[0]
if drop_start_idx != 0:
    df = df.tail(-drop_start_idx).copy()
    df.index -= df.index[0]
    df['time'] -= df.time[0]

lt25_iloc = df[df[col_fit] < 100].index[0]
df = df.head(lt25_iloc).copy()

print('num data points to fit: {}'.format(len(df)))

C_pt0, popt, totaldecay_stderr = test_fit(df)
ACH = popt[0]
print('C_pt0: {}'.format(C_pt0))
print('ACH: {}'.format(ACH))
print('stderr: {}'.format(naturaldecay_stderr))

f = DecayFuncs(0, C_pt0)

plt.scatter(df.time, df[col_fit], label="Original Data")
plt.plot(df.time, f.func(df.time, ACH), 'r-', label="ACH {:0.2f} ±{:0.2f}".format(ACH, totaldecay_stderr))
plt.legend()


For this next part don't forget to change V_r (Volume of room/chamber) to match your value.

In [ ]:
ACH_f = ACH - ACH_vd
# volume of room
V_r = (59. / 12) * (59. / 12) * (72.8 / 12) # this is in cubic feet
CADR = V_r * ACH_f / 60 # units of ACH are 1/h, divide by 60 to convert to 1/minutes so CADR is in cubic feet per minute
CADR_err = V_r * np.sqrt((naturaldecay_stderr/60)**2 + (totaldecay_stderr/60)**2)

print('stderr (ACH): {}'.format(totaldecay_stderr))

fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(18,6))

plt.subplot(121)
plt.scatter(df.time, df[col_fit], label="Original Data")
plt.plot(df.time, f.func(df.time, ACH), 'r-', label="CADR {:0.2f} ±{:0.2f}".format(CADR, CADR_err))
plt.legend()

plt.subplot(122)
plt.scatter(df.time, df[col_fit], label="Original Data")
plt.plot(df.time, f.func(df.time, ACH), 'r-', label="CADR {:0.2f} ±{:0.2f}".format(CADR, CADR_err))
plt.yscale('log')
plt.legend()
plt.show()

Collect results of each trial here in `trials` to get the mean and standard error of the mean

In [ ]:
trials = [94.47, 96.59, 93.92]

print('mean of trials: {}'.format(np.mean(trials)))
print('std error of the mean: {}'.format(scipy.stats.sem(trials)))
print('{:.1f} ±{:.2f}'.format(np.mean(trials), scipy.stats.sem(trials)))